In [1]:
import sys
sys.path.insert(0,'..')

from src.models import *
from src.loss_functions import *
from src.noise import *
from src.metrics import *
from src.plotting import *
from src.generate_data import *
from src.real_data import *
from src.real_data import Metrics, Vectors 
from src.abstain import *

import sklearn
from sklearn.metrics import roc_curve, auc, jaccard_score
import pandas as pd
from supervenn import supervenn

from scipy.stats import bernoulli, spearmanr

from operator import xor

import ast

import os



In [79]:
dataset = "cshock_eicu"
noise_type = "class_conditional"
noise_level = 0.05
fixed_class = 0
fixed_noise = 0.0
max_iter = 1000

X_train, X_test, y_train, y_test, group_train, group_test = load_dataset_splits(dataset, group = "age")

y_train = y_train.astype(int)
y_test = y_test.astype(int)

p_y_x_dict =  calculate_prior(y_train, noise_type = noise_type, group=group_train) #Clean prior

_, T_true = generate_class_conditional_noise(y_train, noise_level, fixed_class, fixed_noise)

u_vec = get_u(y_train, T = T_true, seed= 2024, noise_type = noise_type)
yn_train = flip_labels(y_train, u_vec) #XOR

noise_rates = np.where(y_train == 0, T[0, 1], T[1, 0])

posterior = calculate_posterior(yn_train, T, p_y_x_dict[0])

all_plausible_labels = []

for seed in tqdm(range(1, max_iter+1)):
    y_vec = yn_train
    T= T_true
    epsilon = 0.1
    u_vec = infer_u(y_vec,  noise_type = noise_type, p_y_x_dict = p_y_x_dict,  T = T , seed=seed)

    typical_flag, _ = is_typical(u_vec, p_y_x_dict,   T = T, y_vec = y_vec, noise_type = noise_type, uncertainty_type = "backward", epsilon = epsilon)

    if not typical_flag:
        continue

    flipped_labels = flip_labels(y_vec, u_vec)
    all_plausible_labels.append(flipped_labels)

all_plausible_labels = np.array(all_plausible_labels)

100%|██████████| 1000/1000 [00:00<00:00, 1933.75it/s]


In [80]:
plausible_label_variation = (np.any(all_plausible_labels != all_plausible_labels[0], axis=0)).astype(int)
susceptible_posterior = (posterior > 0).astype(int)
susceptible =  (noise_rates > 0).astype(int)


In [81]:
# Condition 2
# Step 1: Find indices where susceptible_posterior is 1
indices_where_susceptible_posterior = np.where(susceptible_posterior == 1)[0]

# Step 2: Check if plausible_label_variation is 1 for those indices
all_satisfy_condition_2 = np.all(plausible_label_variation[indices_where_susceptible_posterior] == 1)

print("Condition 2 met:", all_satisfy_condition_2)
# Condition 2
# Step 1: Find indices where susceptible_posterior is 1
indices_where_susceptible_posterior = np.where(susceptible_posterior == 1)[0]

# Step 2: Check if plausible_label_variation is 1 for those indices
all_satisfy_condition_2 = np.all(plausible_label_variation[indices_where_susceptible_posterior] == 1)

print("Condition 2 met:", all_satisfy_condition_2)

# Condition 3
# Step 1: Find indices where susceptible is 1 and susceptible_posterior is 0
indices_where_susceptible_and_not_posterior = np.where((susceptible == 1) & (susceptible_posterior == 0))[0]

# Step 2: Check if plausible_label_variation is 0 for those indices
all_satisfy_condition_3 = np.all(plausible_label_variation[indices_where_susceptible_and_not_posterior] == 0)

print("Condition 3 met:", all_satisfy_condition_3)

Condition 2 met: True
Condition 2 met: True
Condition 3 met: True


In [82]:
# Check if there exists any point where susceptible_posterior = 1 and plausible_label_variation = 0
exists_posterior_variation_mismatch = np.any((susceptible_posterior == 1) & (plausible_label_variation == 0))

# Output result
print("Are there any points where p(u | yn[i]) > 0 and plausible_label_variation[i] = 0:", exists_posterior_variation_mismatch)

Are there any points where p(u | yn[i]) > 0 and plausible_label_variation[i] = 0: False


In [93]:
from tqdm import tqdm
import numpy as np

def get_uncertainty(m, max_iter, preds, yn_train, p_y_x_dict,group_train = None, group_test = None, noise_type="class_independent", model_type="LR", T=None, epsilon=0.25, misspecify=False):
    
    typical_count = 0
    y_vec = yn_train
    all_plausible_labels = []
    
    for seed in tqdm(range(1, max_iter+1)):
        u_vec = infer_u(y_vec, group=group_train, noise_type=noise_type, p_y_x_dict=p_y_x_dict, T=T, seed=seed)
        typical_flag, _ = is_typical(u_vec, p_y_x_dict, group=group_train, T=T, y_vec=y_vec, noise_type=noise_type, uncertainty_type="backward", epsilon=epsilon)
        
        if misspecify or noise_type == "group":
            typical_flag = True
            
        if not typical_flag:
            continue
            
        flipped_labels = flip_labels(y_vec, u_vec)
        all_plausible_labels.append(flipped_labels)
    
    all_plausible_labels = np.array(all_plausible_labels)  # Shape: (k, n)
    
    # Calculate Actual Mistake as a vector of mean values for each instance
    actual_mistakes = np.mean(preds != all_plausible_labels, axis=0)  # Shape: (n,)
    

    # Calculate Unanticipated Mistake as a vector of mean values for each instance
    # Expand preds and yn_train to match dimensions for comparison
    preds_expanded = np.expand_dims(preds, axis=0)  # Shape: (1, n)
    yn_train_expanded = np.expand_dims(yn_train, axis=0)  # Shape: (1, n)

    # Case 1: pred == yn_train but pred != all_plausible_labels
    case_1 = (preds_expanded == yn_train_expanded) & (preds_expanded != all_plausible_labels)
    
    # Case 2: pred != yn_train but pred == all_plausible_labels
    case_2 = (preds_expanded != yn_train_expanded) & (preds_expanded == all_plausible_labels)
    
    # Calculate mean unanticipated mistakes for each instance
    unanticipated_mistakes = np.mean(case_1 | case_2, axis=0)  # Shape: (n,)

    return actual_mistakes, unanticipated_mistakes  

In [94]:
m=1000
preds = np.zeros(len(yn_train))
a, b = get_uncertainty(m, max_iter, preds, yn_train, p_y_x_dict, noise_type="class_independent", model_type="LR", T=T_true, epsilon=0.25, misspecify=False)

100%|██████████| 1000/1000 [00:00<00:00, 2024.49it/s]


In [95]:
a[:10], b[:10]

(array([0.039, 1.   , 0.048, 1.   , 0.055, 1.   , 1.   , 0.05 , 1.   ,
        0.053]),
 array([0.039, 0.   , 0.048, 0.   , 0.055, 0.   , 0.   , 0.05 , 0.   ,
        0.053]))